In [20]:
import requests
import s3fs
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq


In [5]:
access_key = "G558I833UBL38WJ6XLPM"
secret_key = "2LjI0sdv4Ghre3VpkGBYnVzWrbHTk757Lu+1zjsC"
token = "eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJHNTU4STgzM1VCTDM4V0o2WExQTSIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzMyNzE4ODIwLCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImNvbGluLmxpY2hlcmVAZW5zYWUuZnIiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiZXhwIjoxNzM0MTgxNTA4LCJmYW1pbHlfbmFtZSI6IkxpY2jDqHJlIiwiZ2l2ZW5fbmFtZSI6IkNvbGluIiwiZ3JvdXBzIjpbIlVTRVJfT05ZWElBIl0sImlhdCI6MTczMzU3NjcwOCwiaXNzIjoiaHR0cHM6Ly9hdXRoLmxhYi5zc3BjbG91ZC5mci9hdXRoL3JlYWxtcy9zc3BjbG91ZCIsImp0aSI6IjFjMjU5YmIyLTZjMDAtNGRkNS1iZGFhLThjMWU4N2MzZDE1MSIsIm5hbWUiOiJDb2xpbiBMaWNow6hyZSIsInBvbGljeSI6InN0c29ubHkiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJjbGljaGVyZSIsInJlYWxtX2FjY2VzcyI6eyJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLXNzcGNsb3VkIl0sInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgZ3JvdXBzIGVtYWlsIiwic2lkIjoiOGJlYTQ2MzctYTUzMi00YTY2LTkxOTctMDY3NDliYTMxZDVjIiwic3ViIjoiY2RkOGM5MzYtNWUxYy00OTE4LWIxMmItMjgzNjY1NzY5YTFlIiwidHlwIjoiQmVhcmVyIn0.iZazPSQeCUZSJ-QSaQduhl_kNl1PeNU_i4o09zGSKPdHfHQVbMdp5bmWLaVmAUE2Rrs3HwpTnk_tFaga-wpC8g"


In [37]:
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"}, key=access_key,
        secret=secret_key, token=token)

# Testez la connexion en listant les fichiers d’un bucket
try:
    files = fs.ls("clichere")  # Remplacez par le nom de votre bucket
    print("Connexion réussie. Fichiers disponibles :", files)
except Exception as e:
    print("Erreur lors de la connexion :", e)

def read_csv_from_s3(path: str, columns_to_select: list, dtype_spec: dict, encoding: str = "utf-8") -> pd.DataFrame:
  
    with fs.open(path, mode="r", encoding=encoding) as file:
        df = pd.read_csv(file, usecols=columns_to_select, dtype=dtype_spec)
    
    return df


Connexion réussie. Fichiers disponibles : ['clichere/diffusion']


1. Revenus par habitant et population par commune

In [33]:
# Définir les colonnes à sélectionner et les types de données
columns_to_select = ["codecommune", "nomcommune", "pop2022", "revmoy2022"]
dtype_spec = {'codecommune': 'str'}

# Lire le fichier CSV depuis S3
path_revcommunes = "clichere/diffusion/revcommunes.csv"
revcommunes = read_csv_from_s3(path_revcommunes, columns_to_select, dtype_spec)

# Afficher un aperçu des données lues ainsi que les statistiques descriptives
print(revcommunes.head())

print(revcommunes.describe())

  codecommune             nomcommune  pop2022  revmoy2022
0       01001  ABERGEMENT-CLEMENCIAT      747   21615.654
1       01002    ABERGEMENT-DE-VAREY      288   16945.766
2       01004      AMBERIEU-EN-BUGEY    14375   18423.111
3       01005    AMBERIEUX-EN-DOMBES     1717   20996.125
4       01006                AMBLEON      116   18075.986
            pop2022     revmoy2022
count  3.666100e+04   34746.000000
mean   1.925007e+03   17051.483011
std    1.495215e+04    5197.378499
min    0.000000e+00    3350.483900
25%    1.820000e+02   14012.091750
50%    4.290000e+02   16322.690500
75%    1.118000e+03   19126.264250
max    2.118266e+06  126903.910000


2. Taux de diplome par commune

In [34]:
# Définir les colonnes à sélectionner et les types de données
columns_to_select = ["codecommune","pbac2022", "psup2022"]
dtype_spec = {'codecommune': 'str'}

# Lire le fichier CSV depuis S3
path_dipcommunes = "clichere/diffusion/diplomescommunes.csv"
dipcommunes = read_csv_from_s3(path_dipcommunes, columns_to_select, dtype_spec)

# Afficher un aperçu des données lues ainsi que les statistiques descriptives
print(dipcommunes.head())

print(dipcommunes.describe())


  codecommune  pbac2022  psup2022
0       01001  0.447140  0.232236
1       01002  0.672414  0.086207
2       01003       NaN       NaN
3       01004  0.461044  0.289937
4       01005  0.337925  0.168558
           pbac2022      psup2022
count  35222.000000  35222.000000
mean       0.460833      0.274160
std        0.199317      0.174347
min        0.000000      0.000000
25%        0.333333      0.157895
50%        0.451748      0.259494
75%        0.575221      0.372092
max        1.000000      1.000000


3. Fusionner les bases de données

In [35]:
rev_dip_pop_comm = pd.merge(revcommunes, dipcommunes, on='codecommune', how='inner')


print(rev_dip_pop_comm)

      codecommune             nomcommune  pop2022  revmoy2022  pbac2022  \
0           01001  ABERGEMENT-CLEMENCIAT      747   21615.654  0.447140   
1           01002    ABERGEMENT-DE-VAREY      288   16945.766  0.672414   
2           01004      AMBERIEU-EN-BUGEY    14375   18423.111  0.461044   
3           01005    AMBERIEUX-EN-DOMBES     1717   20996.125  0.337925   
4           01006                AMBLEON      116   18075.986  1.000000   
...           ...                    ...      ...         ...       ...   
36627       95676     VILLERS-EN-ARTHIES      529   20655.250  0.533520   
36628       95678          VILLIERS-ADAM      927   26113.932  0.891641   
36629       95680        VILLIERS-LE-BEL    27033   11837.685  0.436124   
36630       95682        VILLIERS-LE-SEC      248   18516.314  0.270115   
36631       95690    WY-DIT-JOLY-VILLAGE      375   25304.949  0.871795   

       psup2022  
0      0.232236  
1      0.086207  
2      0.289937  
3      0.168558  
4      0.

In [75]:
output_path1 = 'clichere/diffusion/rev_dip_pop_comm.parquet'
output_path2 = 'clichere/diffusion/revcommunes.parquet'
output_path3 = 'clichere/diffusion/dipcommunes.parquet'

# Exporter le DataFrame en format Parquet sur S3
rev_dip_pop_comm.to_parquet(output_path1, engine='pyarrow', compression='snappy', filesystem=fs)
revcommunes.to_parquet(output_path2, engine='pyarrow', compression='snappy', filesystem=fs)
dipcommunes.to_parquet(output_path3, engine='pyarrow', compression='snappy', filesystem=fs)


In [36]:
# Calculer le nombre de valeurs manquantes (NA) par colonne
na_counts = rev_dip_pop_comm.isna().sum()
na_percentage = rev_dip_pop_comm.isna().mean() * 100

# Afficher les résultats
print(na_counts)
print(na_percentage)

codecommune       0
nomcommune        7
pop2022           0
revmoy2022     1912
pbac2022       1410
psup2022       1410
dtype: int64
codecommune    0.000000
nomcommune     0.019109
pop2022        0.000000
revmoy2022     5.219480
pbac2022       3.849094
psup2022       3.849094
dtype: float64


4. Récupération de la base de données de travail

In [26]:
dataset = pq.ParquetDataset('aayrapetyan/diffusion/final_table.parquet', filesystem=s3)
final_dataset = dataset.read().to_pandas()

# Afficher les premières lignes
print(final_dataset.head())

   total_voyageurs_2022 code_postal          nom_gare       uic  lon_gare  \
0                 40825       44170          Abbaretz  87481614 -1.524416   
1                 80648       62121   Achiet-le-Grand  87342048  2.780168   
2                 11325       30220     Aigues-Mortes  87775858  4.191210   
3                128969       80250    Ailly-sur-Noye  87313221  2.364190   
4                126465       73210  Aime - La Plagne  87741769  6.648646   

    lat_gare codecommune              nomcommune  pop2022  revmoy2022  \
0  47.554643       44170  ST-JULIEN-DE-VOUVANTES   1017.0  12806.5490   
1  50.131752       62121               BEUGNATRE    243.0   8432.3916   
2  43.570901       30220                ROQUEDUR    312.0   7633.2280   
3  49.754480       80250                 DOMVAST    365.0  14541.2230   
4  45.554400       73210                 PUYGROS    417.0  17936.0200   

   pbac2022  psup2022  
0  0.321522  0.164042  
1  0.532110  0.128440  
2  0.669456  0.422594  
3 

In [41]:
select_columns = ['pop2022', 'revmoy2022', 'pbac2022', 'psup2022']

print(rev_dip_pop_comm[select_columns].describe())
print(final_dataset[select_columns].describe())

             pop2022     revmoy2022      pbac2022      psup2022
count   36632.000000   34720.000000  35222.000000  35222.000000
mean     1829.919387   17049.742626      0.460833      0.274160
std      9954.647803    5194.780298      0.199317      0.174347
min         0.000000    3350.483900      0.000000      0.000000
25%       182.000000   14011.968000      0.333333      0.157895
50%       428.000000   16321.857000      0.451748      0.259494
75%      1116.000000   19124.010750      0.575221      0.372092
max    875791.000000  126903.910000      1.000000      1.000000
             pop2022     revmoy2022     pbac2022     psup2022
count    2938.000000    2938.000000  2938.000000  2938.000000
mean     9941.618788   18538.509528     0.476672     0.304576
std     33823.934961    6817.612670     0.165917     0.155725
min        23.000000    5536.604500     0.000000     0.000000
25%       513.000000   14877.651000     0.367915     0.202236
50%      1479.000000   17182.293000     0.474320    